In [1]:
!pip install catalyst
!pip install -U git+https://github.com/albu/albumentations --no-cache-dir
!pip install efficientnet_pytorch
!pip install timm
!pip install torchtoolbox

  Cloning https://github.com/albu/albumentations to /tmp/pip-req-build-hliqne2_
  Running command git clone --filter=blob:none -q https://github.com/albu/albumentations /tmp/pip-req-build-hliqne2_
  Resolved https://github.com/albu/albumentations to commit 880c1aaed10ab74cfe851496a476c1c34cadcd0f
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=2fbb212ccb5e64e9902839b7a106253b1b7c28e4db3b08b51cca39c505f51ef6
  Stored in directory: /root/.cache/pip/wheels/0e/cc/b2/49e74588263573ff778da58cc99b9c6349b496636a7e165be6
Successfully built efficientnet-pytorch
     |████████████████████████████████| 376 kB 516 kB/s            
     |████████████████████████████████| 84 kB 617 kB/s            


In [2]:
# !pip install timm

In [3]:
import os
from torch import nn, optim
from torch.utils.data import DataLoader
from catalyst import dl, utils
from catalyst.contrib.datasets import MNIST
import numpy as np
import cv2
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torch.utils.data import Dataset as BaseDataset
import albumentations
from albumentations import pytorch as AT
import torch
import torch.nn as nn
import timm
from efficientnet_pytorch import EfficientNet
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, accuracy_score, f1_score
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torchmetrics
from albumentations.core.composition import Compose, OneOf
from albumentations.augmentations.transforms import CLAHE, GaussNoise, ISONoise
from albumentations.pytorch import ToTensorV2
import pytorch_lightning as pl
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning import Callback
from pytorch_lightning.loggers import CSVLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from torchtoolbox.transform import ImageNetPolicy
import matplotlib.pyplot as plt
img_size = 224

In [4]:
class CFG:
    seed = 42
    model_name = 'efficientnet-b3'
    pretrained = True
    img_size = 224
    num_classes = 7
    lr = 3e-4
    min_lr = 1e-6
    t_max = 20
    num_epochs = 40
    batch_size = 16
    accum = 1
    precision = 16
    n_fold = 5
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



In [5]:
class Dataset(BaseDataset):
    def __init__(
            self, 
            data_dir=None,
            augmentation=None, 
    ):
        # self.ids = os.listdir(images_dir)
        # self.images_fps = [os.path.join(images_dir, image_id) for image_id in self.ids]
        # self.masks_fps = [os.path.join(masks_dir, image_id) for image_id in self.ids]
        
        # self.ids = []
        self.images = []
        for emo_dir in os.listdir(data_dir):
            cur_dir = os.path.join(data_dir, emo_dir)
            for file in os.listdir(cur_dir):
                self.images.append((emo_dir, os.path.join(cur_dir, file)))
#         try:
    #         except:
#                 print(data_fer_dir)
        labels_tr = ['ambulance', 'police', 'evacuator', 'fire', 'car']
        self.map_labels = {name : ind for ind, name in enumerate(labels_tr)}
        self.augmentation = augmentation
    
    def get_images_from_folder(self, path):
        return os.listdir(path)

    def __getitem__(self, i):
        
        # read data
        cur_pair = self.images[i]
        image = cv2.imread(cur_pair[1])
#         if 'train_fer' not in cur_pair[1]:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#         else:
#             image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
        
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = self.make_square(image)
        label = cur_pair[0]

        # apply augmentations
        if self.augmentation:
            image = self.augmentation(image=image)['image']
            
        
        return image, self.map_labels[label]
    

    def __len__(self):
        return len(self.images)
    def make_square(self, img):
        sz = max(img.shape[0], img.shape[1])
        left = (sz - img.shape[1]) // 2
        right = sz - img.shape[1] - left
        top = (sz - img.shape[0]) // 2
        bottom = sz - img.shape[0] - top
#         print(top, bottom, left, right)
        return cv2.copyMakeBorder(img, top + 10, bottom + 10, left + 10, right + 10, cv2.BORDER_CONSTANT, None, value=0)

In [6]:
def make_square(img):
    sz = max(img.shape[0], img.shape[1])
    left = (sz - img.shape[1]) // 2
    right = sz - img.shape[1] - left
    top = (sz - img.shape[0]) // 2
    bottom = sz - img.shape[0] - top
#         print(top, bottom, left, right)
    return cv2.copyMakeBorder(img, top + 10, bottom + 10, left + 10, right + 10, cv2.BORDER_CONSTANT, None, value=0)

In [7]:
data_transform = {
    'val': albumentations.Compose([
        albumentations.Resize(img_size, img_size),
        albumentations.Normalize(),
#         albumentations.ToFloat(),
#         albumentations.ToGray(always_apply=True, p=1.0),
        AT.ToTensorV2()
        ]),
    'train': albumentations.Compose([
        albumentations.Resize(img_size, img_size),
        albumentations.HorizontalFlip(0.5),
        albumentations.RGBShift(),
        albumentations.Blur(),
        albumentations.OneOf([
            albumentations.GaussNoise(),
            albumentations.ISONoise()]),
#         albumentations.ElasticTransform(),
        albumentations.CoarseDropout(p=1),
        albumentations.Rotate(always_apply=False, p=1.0, limit=(-10, 10), interpolation=0, border_mode=0, value=(0, 0, 0), mask_value=None),
#         albumentations.ShiftScaleRotate(p=0.5),
#         albumentations.RandomBrightnessContrast(p=0.5),
#         ImageNetPolicy,
        
        albumentations.Normalize(),
#         albumentations.ToFloat(),
#         albumentations.ToGray(always_apply=True, p=1.0),
        AT.ToTensorV2()
        ]),
    'test': albumentations.Compose([
        albumentations.Resize(img_size, img_size),
        albumentations.Normalize(),
#         albumentations.ToFloat(),
#         albumentations.ToGray(always_apply=True, p=1.0),
        AT.ToTensorV2()
        ])
}



In [8]:
# helper function for data visualization
def visualize(images):
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(16, 5))
    cur = 0
    for i, (image, name) in enumerate(images):
        plt.title(name)
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.imshow(image.numpy().transpose(1, 2, 0))
        cur += 1
            
    plt.show()

In [9]:
class CustomResnext(nn.Module):
    def __init__(self, model_name='resnext50d_32x4d', pretrained=True):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained, num_classes=5)

    def forward(self, x):
        x = self.model(x)
        return x

In [10]:
model = CustomResnext()
# model.load_state_dict(torch.load('./logs/train.7.pth')['model_state_dict'])
# model.to("cuda")

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/resnext50d_32x4d-103e99f8.pth" to /root/.cache/torch/hub/checkpoints/resnext50d_32x4d-103e99f8.pth


In [11]:
# model

In [12]:
# from catalyst.contrib.schedulers import MultiStepLR, ReduceLROnPlateau

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=3e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')
train_data = Dataset(data_dir='../input/dataset-dtp-with-evacuator/dataset_dtp_vid_with_evacuator_new1/train', augmentation=data_transform['train'])
valid_data = Dataset(data_dir='../input/dataset-dtp-with-evacuator/dataset_dtp_vid_with_evacuator_new1/test', augmentation=data_transform['test'])
loaders = {
    "train": DataLoader(train_data, batch_size=32, shuffle=True),
    "valid": DataLoader(valid_data, batch_size=32),
}

In [13]:
!nvidia-smi

Sat Jan 15 13:09:49 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.119.04   Driver Version: 450.119.04   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    28W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [14]:
# optimizer.param_groups[0]['lr'] = 1e-5

In [15]:
runner = dl.SupervisedRunner(
    input_key="features", output_key="logits", target_key="targets", loss_key="loss"
)

# model training
runner.train(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    loaders=loaders,
    num_epochs=20,
    callbacks=[
        dl.AccuracyCallback(input_key="logits", target_key="targets", topk_args=[1]),
        dl.PrecisionRecallF1SupportCallback(
            input_key="logits", target_key="targets", num_classes=5),
        dl.SchedulerCallback(loader_key="valid", metric_key="loss"),
        dl.CheckpointCallback(
            logdir="./logs", loader_key="valid", metric_key="loss", minimize=True
        ),
    ],
    logdir="./logs",
    valid_loader="valid",
    valid_metric="loss",
#     minimize_valid_metric=True,
    verbose=True,
    load_best_on_end=True,
)


1/20 * Epoch (train):   0%|          | 0/153 [00:00<?, ?it/s]

train (1/20) accuracy01: 0.8909128121162504 | accuracy01/std: 0.10212709970902326 | f1/_macro: 0.8918118374345181 | f1/_micro: 0.8909078121443115 | f1/_weighted: 0.891065291857228 | loss: 0.3337479339843154 | loss/mean: 0.3337479339843154 | loss/std: 0.25477147302865744 | lr: 0.0003 | momentum: 0.9 | precision/_macro: 0.8917451971969541 | precision/_micro: 0.8909128121162505 | precision/_weighted: 0.8915623916170279 | recall/_macro: 0.8922115210851942 | recall/_micro: 0.8909128121162505 | recall/_weighted: 0.8909128121162505


1/20 * Epoch (valid):   0%|          | 0/30 [00:00<?, ?it/s]

valid (1/20) accuracy01: 0.9353813559322035 | accuracy01/std: 0.054266659378579024 | f1/_macro: 0.9376515431672139 | f1/_micro: 0.9353763559589303 | f1/_weighted: 0.9350858995616218 | loss: 0.2102375808250854 | loss/mean: 0.2102375808250854 | loss/std: 0.14999378348056105 | lr: 0.0003 | momentum: 0.9 | precision/_macro: 0.9317642423576137 | precision/_micro: 0.9353813559322034 | precision/_weighted: 0.9382066441352599 | recall/_macro: 0.9469290212711264 | recall/_micro: 0.9353813559322034 | recall/_weighted: 0.9353813559322033
* Epoch (1/20) lr: 0.0003 | momentum: 0.9


2/20 * Epoch (train):   0%|          | 0/153 [00:00<?, ?it/s]

...

valid (19/20) accuracy01: 0.9671610169491527 | accuracy01/std: 0.04087771803609542 | f1/_macro: 0.9684763024375485 | f1/_micro: 0.9671560169750012 | f1/_weighted: 0.967338162481802 | loss: 0.13746097119933512 | loss/mean: 0.13746097119933512 | loss/std: 0.20048591494518678 | lr: 2.9999999999999997e-05 | momentum: 0.9 | precision/_macro: 0.963126340463492 | precision/_micro: 0.9671610169491526 | precision/_weighted: 0.968814737798612 | recall/_macro: 0.9753614806246386 | recall/_micro: 0.9671610169491526 | recall/_weighted: 0.9671610169491526
* Epoch (19/20) lr: 2.9999999999999997e-05 | momentum: 0.9


20/20 * Epoch (train):   0%|          | 0/153 [00:00<?, ?it/s]

train (20/20) accuracy01: 0.9973393368808843 | accuracy01/std: 0.009427429865519686 | f1/_macro: 0.9974070007695242 | f1/_micro: 0.9973343369059506 | f1/_weighted: 0.9973351733032034 | loss: 0.006450589975302888 | loss/mean: 0.006450589975302888 | loss/std: 0.014813644079200087 | lr: 2.9999999999999997e-05 | momentum: 0.9 | precision/_macro: 0.9974928681446584 | precision/_micro: 0.9973393368808842 | precision/_weighted: 0.9973425737246845 | recall/_macro: 0.9973328591979822 | recall/_micro: 0.9973393368808842 | recall/_weighted: 0.9973393368808843


20/20 * Epoch (valid):   0%|          | 0/30 [00:00<?, ?it/s]

valid (20/20) accuracy01: 0.9671610169491527 | accuracy01/std: 0.04246814694828753 | f1/_macro: 0.9680040784766284 | f1/_micro: 0.9671560169750012 | f1/_weighted: 0.9673293671159213 | loss: 0.1365273969183317 | loss/mean: 0.1365273969183317 | loss/std: 0.1948009976463055 | lr: 2.9999999999999997e-05 | momentum: 0.9 | precision/_macro: 0.9624388426738388 | precision/_micro: 0.9671610169491526 | precision/_weighted: 0.9687328176965959 | recall/_macro: 0.9749646552278131 | recall/_micro: 0.9671610169491526 | recall/_weighted: 0.9671610169491526
* Epoch (20/20) lr: 2.9999999999999997e-05 | momentum: 0.9
Top best models:
logs/train.3.pth	0.0996


In [16]:
# mod = runner.bes

In [17]:
# import os
# os.listdir('../input/emotion-recognition-dataset/RDFKDEF_yolo_bounding_box30px/rfd_cropped/test_many_photos_per_human')

In [18]:
# print(torch.load('./logs/train.7.pth'))

In [19]:
# pretrained_model = MyEffnet()
# # pretrained_model = torch.load('./logs/train.7.pth')
# pretrained_model.load_state_dict(torch.load('./logs/train.7.pth')['model_state_dict'])
# pretrained_model.to('cuda')
# pretrained_model.eval()

In [20]:
# model.cuda()
# model = EmotionsLightning(MyEffnet())
# model = EmotionsLightning(model=MyEffnet()).load_from_checkpoint(checkpoint_callback.best_model_path, model=MyEffnet())
# model.to('cuda')
# model = MyEffnet()
# model_cfg = torch.load(checkpoint_callback.best_model_path)
# checkpoint_callback.best_model_path
# model.eval()
# model = MyEffnet()
# model.load_state_dict(checkpoint_callback.best_model_path, model_cfg['state_dict'])
# model_cfg['state_dict']
# # print(model)
# model.to('cuda')

In [21]:
# model.load_state_dict(utils.get_averaged_weights_by_path_mask(logdir="./logs", path_mask="*.pth"))

In [23]:
torch.save(model, 'best_model.pth')
torch.save(model.state_dict(), 'best_model_dict.pth')